In [25]:
from scrapping import *
from fonctions_preprocess import *

In [26]:
B = pd.read_csv("/home/onyxia/work/Projet-python/Fbref_alex/SOCCER_241223.csv", index_col=0)
C = pd.read_csv("/home/onyxia/work/Projet-python/Fbref_alex/recup_matchweek.csv", index_col=0)

In [27]:
mapping_equipe = {
    'Nimes': 'Nîmes',
    'Paris S-G': 'Paris Saint Germain',
    'Saint Etienne': 'Saint-Étienne'
}
pd.set_option('display.max_columns', None)

In [28]:
from datetime import datetime

ajd = datetime.now()

B = preprocess_initial(B, mapping_equipe=mapping_equipe)
C = preprocess_initial(C, mapping_equipe)
B = renommer_colonnes(B)
C = renommer_colonnes(C)
B = columns_to_keep(B)
C = columns_to_keep(C)
B = preprocess_variables(B)
resultat = pd.concat([C, B], sort=False).reset_index(drop= True)
resultat = preparation_model(B)
resultat = preprocess_data(B)


In [29]:
a=modelisation(resultat, datetime(2023, 12, 16, 21, 0, 0))
a["Result"]=np.NaN




In [30]:
# Fusionner les DataFrames
a = a.merge(resultat[['MatchID', 'Result']], on='MatchID', how='left', suffixes=('', '_from_resultat'))

# Mettre à jour la colonne 'Result' dans a
a['Result'].update(a['Result_from_resultat'])

# Supprimer la colonne supplémentaire
a.drop(columns=['Result_from_resultat'], inplace=True)


/tmp/ipykernel_288800/2344193416.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['D' 'W_Home' 'W_Away' 'W_Away' 'D' 'W_Away' 'W_Home' 'W_Home' 'W_Home'
 'W_Home' 'W_Home' 'D' 'W_Home' 'W_Away' 'W_Away']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  a['Result'].update(a['Result_from_resultat'])


In [31]:
a

,DateTime,Comp,Saison,Round,Day,Team Home,Team Away,Result,Predicted_Result,Prediction_Probability,MatchID
0,2023-12-17 20:45:00,Ligue 1,2023-2024,16,Sun,Lille,Paris Saint Germain,D,W_Away,0.53,2023-12-17 20:45:00-Lille-vs-Paris Saint Germain
1,2023-12-17 17:05:00,Ligue 1,2023-2024,16,Sun,Marseille,Clermont Foot,W_Home,W_Home,0.44,2023-12-17 17:05:00-Clermont Foot-vs-Marseille
2,2023-12-17 13:00:00,Ligue 1,2023-2024,16,Sun,Nantes,Brest,W_Away,W_Away,0.40,2023-12-17 13:00:00-Brest-vs-Nantes
3,2023-12-17 15:00:00,Ligue 1,2023-2024,16,Sun,Metz,Montpellier,W_Away,W_Home,0.41,2023-12-17 15:00:00-Metz-vs-Montpellier
4,2023-12-17 15:00:00,Ligue 1,2023-2024,16,Sun,Toulouse,Rennes,D,W_Away,0.46,2023-12-17 15:00:00-Rennes-vs-Toulouse
5,2023-12-17 15:00:00,Ligue 1,2023-2024,16,Sun,Lorient,Strasbourg,W_Away,D,0.40,2023-12-17 15:00:00-Lorient-vs-Strasbourg
6,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Paris Saint Germain,Metz,W_Home,W_Home,0.72,2023-12-20 21:00:00-Metz-vs-Paris Saint Germain
7,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Nice,Lens,W_Home,W_Home,0.45,2023-12-20 21:00:00-Lens-vs-Nice
8,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Brest,Lorient,W_Home,W_Home,0.49,2023-12-20 21:00:00-Brest-vs-Lorient
9,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Reims,Le Havre,W_Home,D,0.44,2023-12-20 21:00:00-Le Havre-vs-Reims


In [7]:
selected_columns = ["Result", "DateTime"] + [col for col in resultat.columns if col.endswith(('Lag1_Home', 'Lag1_Away', 'Lag_Home', 'Lag_Away'))]
cutoff_date = datetime(2023, 12, 20, 21, 0, 0)
X = resultat[selected_columns].copy() 
X["Result"] = X['Result'].astype('category')

# Séparation des données
X_train = X[resultat['DateTime'] <= cutoff_date].dropna()
y_train = X_train["Result"]
X_train.drop(columns=['Result', 'DateTime'], errors='ignore', inplace=True)
X_test = X[resultat['DateTime'] > cutoff_date].drop(columns=['Result', 'DateTime'], errors='ignore')


In [9]:
X_test

,Points_Cum_Lag1_Home,GD_Cum_Lag1_Home,GF_Cum_Lag1_Home,GA_Cum_Lag1_Home,Classement_Lag1_Home,CumulativeWins_Lag1_Home,CumulativeDraws_Lag1_Home,CumulativeLosses_Lag1_Home,Moyenne_Total Shots_Lag_Home,Moyenne_Shots on Target_Lag_Home,Moyenne_Shots on Target %_Lag_Home,Moyenne_Goals per Shot_Lag_Home,Moyenne_Total Touches_Lag_Home,Moyenne_Touches in Defensive Penalty Area_Lag_Home,Moyenne_Touches in Defensive Third_Lag_Home,Moyenne_Touches in Midfield Third_Lag_Home,Moyenne_Touches in Attacking Third_Lag_Home,Moyenne_Touches in Attacking Penalty Area_Lag_Home,Moyenne_Dribbles Attempted_Lag_Home,Moyenne_Successful Dribbles_Lag_Home,Moyenne_Successful Dribble %_Lag_Home,Moyenne_Total Carries_Lag_Home,Moyenne_Total Carry Distance_Lag_Home,Moyenne_Progressive Carry Distance_Lag_Home,Moyenne_Progressive Carries_Lag_Home,Moyenne_Carries into Final Third_Lag_Home,Moyenne_Carries into Penalty Area_Lag_Home,Moyenne_Tackles_Lag_Home,Moyenne_Tackles Won_Lag_Home,Moyenne_Tackles in Defensive Third_Lag_Home,Moyenne_Tackles in Midfield Third_Lag_Home,Moyenne_Tackles in Attacking Third_Lag_Home,Moyenne_Dribblers Tackled_Lag_Home,Moyenne_Total Dribbles Against_Lag_Home,Moyenne_Defensive Dribblers Win %_Lag_Home,Moyenne_Interceptions_Lag_Home,Moyenne_Errors Leading to Goal_Lag_Home,Moyenne_Key Passes_Lag_Home,Moyenne_Passes Completed_Lag_Home,Moyenne_Passes Attempted_Lag_Home,Moyenne_Passes into Final Third_Lag_Home,Moyenne_Progressive Passes_Lag_Home,Moyenne_Shots on Target Against_Lag_Home,Moyenne_Keeper Saves_Lag_Home,Moyenne_Keeper Save Percentage_Lag_Home,Points_Cum_Lag1_Away,GD_Cum_Lag1_Away,GF_Cum_Lag1_Away,GA_Cum_Lag1_Away,Classement_Lag1_Away,CumulativeWins_Lag1_Away,CumulativeDraws_Lag1_Away,CumulativeLosses_Lag1_Away,Moyenne_Total Shots_Lag_Away,Moyenne_Shots on Target_Lag_Away,Moyenne_Shots on Target %_Lag_Away,Moyenne_Goals per Shot_Lag_Away,Moyenne_Total Touches_Lag_Away,Moyenne_Touches in Defensive Penalty Area_Lag_Away,Moyenne_Touches in Defensive Third_Lag_Away,Moyenne_Touches in Midfield Third_Lag_Away,Moyenne_Touches in Attacking Third_Lag_Away,Moyenne_Touches in Attacking Penalty Area_Lag_Away,Moyenne_Dribbles Attempted_Lag_Away,Moyenne_Successful Dribbles_Lag_Away,Moyenne_Successful Dribble %_Lag_Away,Moyenne_Total Carries_Lag_Away,Moyenne_Total Carry Distance_Lag_Away,Moyenne_Progressive Carry Distance_Lag_Away,Moyenne_Progressive Carries_Lag_Away,Moyenne_Carries into Final Third_Lag_Away,Moyenne_Carries into Penalty Area_Lag_Away,Moyenne_Tackles_Lag_Away,Moyenne_Tackles Won_Lag_Away,Moyenne_Tackles in Defensive Third_Lag_Away,Moyenne_Tackles in Midfield Third_Lag_Away,Moyenne_Tackles in Attacking Third_Lag_Away,Moyenne_Dribblers Tackled_Lag_Away,Moyenne_Total Dribbles Against_Lag_Away,Moyenne_Defensive Dribblers Win %_Lag_Away,Moyenne_Interceptions_Lag_Away,Moyenne_Errors Leading to Goal_Lag_Away,Moyenne_Key Passes_Lag_Away,Moyenne_Passes Completed_Lag_Away,Moyenne_Passes Attempted_Lag_Away,Moyenne_Passes into Final Third_Lag_Away,Moyenne_Progressive Passes_Lag_Away,Moyenne_Shots on Target Against_Lag_Away,Moyenne_Keeper Saves_Lag_Away,Moyenne_Keeper Save Percentage_Lag_Away


In [39]:
resultat[(resultat["Saison"] == "2023-2024") & (resultat["Round"] == 17)][["DateTime", "Comp", "Saison", "Round", "Day","Team Home", "Team Away", "Result",  "MatchID"]]

,DateTime,Comp,Saison,Round,Day,Team Home,Team Away,Result,MatchID
1806,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Brest,Lorient,W_Home,2023-12-20 21:00:00-Brest-vs-Lorient
1815,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Clermont Foot,Rennes,W_Away,2023-12-20 21:00:00-Clermont Foot-vs-Rennes
1857,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Lyon,Nantes,W_Home,2023-12-20 21:00:00-Lyon-vs-Nantes
1890,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Montpellier,Marseille,D,2023-12-20 21:00:00-Marseille-vs-Montpellier
1908,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Nice,Lens,W_Home,2023-12-20 21:00:00-Lens-vs-Nice
1917,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Paris Saint Germain,Metz,W_Home,2023-12-20 21:00:00-Metz-vs-Paris Saint Germain
1925,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Reims,Le Havre,W_Home,2023-12-20 21:00:00-Le Havre-vs-Reims
1943,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Strasbourg,Lille,W_Home,2023-12-20 21:00:00-Lille-vs-Strasbourg
1951,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Toulouse,Monaco,W_Away,2023-12-20 21:00:00-Monaco-vs-Toulouse


In [9]:
modelisation(resultat, ajd).to_csv("futur_result.csv")

/home/onyxia/work/Projet-python/Fbref_alex/TestAutom/fonctions_preprocess.py:442: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y_train = X_train[df['DateTime'] <= cutoff_date]["Result"]


In [33]:
A = pd.read_csv("/home/onyxia/work/Projet-python/Fbref_alex/TestAutom/futur_result.csv", index_col=0)
A["DateTime"] = pd.to_datetime(A["DateTime"])
derniere_date = A["DateTime"].max()

if derniere_date <= ajd:

    base_actuelle = pd.read_csv("/home/onyxia/work/Projet-python/Fbref_alex/SOCCER_241223.csv", index_col=0)
    data = scrape_latest_ligue1_data()
    base_supplémentaire = data[0]
    base_updated = add_new_matches(base_initiale=base_actuelle, base_nouvelle=base_supplémentaire)
    base_updated.to_csv("/home/onyxia/work/Projet-python/Fbref_alex/SOCCER_241223.csv")

    recup_matchweek = data[1]

    recup_matchweek=find_futur_matchweeks(recup_matchweek, mapping_equipe)

    if recup_matchweek == None:
        print("Pas de nouveaux matchs à récupérer")
    else:
        
        recup_matchweek2 = recup_matchweek.copy()
        recup_matchweek2 = preprocess_initial(recup_matchweek2, mapping_equipe)
        recup_matchweek2 = renommer_colonnes(recup_matchweek2)
        recup_matchweek2 = columns_to_keep(recup_matchweek2)

        base_updated2 = base_updated.copy()
        base_updated2 = preprocess_initial(base_updated2, mapping_equipe)
        base_updated2 = renommer_colonnes(base_updated2)
        base_updated2 = columns_to_keep(base_updated2)
        base_updated2 = preprocess_variables(base_updated2)



        resultat = pd.concat([base_updated2, recup_matchweek2], sort=False).reset_index(drop= True)
        resultat = preparation_model(resultat)
        resultat = preprocess_data(resultat, ajd)

        # Fusionner les DataFrames
        A = A.merge(resultat[['MatchID', 'Result']], on='MatchID', how='left', suffixes=('', '_from_resultat'))
        A['Result'].update(A['Result_from_resultat'])
        A.drop(columns=['Result_from_resultat'], inplace=True)

        resultat = modelisation(resultat, ajd)
        A = pd.concat([A, resultat]).sort_values(by="DateTime").reset_index(drop= True)

        A.to_csv("futur_result.csv")
        

        


    #base_updated.to_csv("/home/onyxia/work/Projet-python/Fbref_alex/SOCCER_241223.csv")
    #base_updated.to_csv("Fbref_alex/SOCCER_241223.csv")
    


    #recup_matchweek.to_csv("/home/onyxia/work/Projet-python/Fbref_alex/recup_matchweek.csv")
    #recup_matchweek.to_csv("Fbref_alex/recup_matchweek.csv")
    

IndexError: list index out of range

In [36]:
scrape_ligue1_data()

IndexError: list index out of range